# Homework 5 - Simulating Polymer Melts

So far, we've only looked at simulations of individual particles, but now that we've learned some of the basics of MD, we can apply our knowledge to more complex systems, like polymer melts. *(yay we love polymers!)* The instructions for this assignment are at the end of this notebook.

As always, we start by importing our packages.

In [ ]:
import numpy as np
import hoomd
import gsd.hoomd

We are now going to load in our input configuration file. Make sure that you use `gen-init-config.ipynb` to create your initial configuration file. For these simulations, we have one particle type, `'typeA'`. We now also have bonds in our system! Bonds have properties similar to particles, there are bond types (`'A-A'` for this simulation), bond type ids, and bond groups. All of this was set up in the initial configuration file.  

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)
sim.create_state_from_gsd('init_config.gsd')
integrator = hoomd.md.Integrator(dt=0.005)

Now we can set up the simulation. We are going to use a Lennard-Jones pair potential as usual, with a cutoff of $2^{1/6}\sigma$ to create our excluded volume interactions. 

In [ ]:
cell = hoomd.md.nlist.Cell(buffer=0.4)
lj = hoomd.md.pair.LJ(nlist=cell, r_cut=2**(1/6))
lj.params['typeA'] = dict(epsilon=1, sigma=1)
integrator.forces.append(lj)

Now, we need to set up the bond force for our bonded interactions. Polymers are commonly modeled in coarse-grained simulations using a bead-spring model. This model uses beads (spherical particles) to represent groups of monomers connected by springs with a Finitely Extensible Non-linear Elastic (FENE) potential. HOOMD extends this potential to include an excluded volume interaction term, also known as the Weeks-Chandler-Anderson (WCA) potential (this is just the LJ potential truncated at $2^{1/6}\sigma$). 
$$U_{FENE}(r)=-0.5KR_0^2ln\left[1-\left(\frac{r}{R_0}\right)^2\right]+4\epsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right]$$
with $K$ as the spring constant, and $R_0$ as the maximum bond length. We are going to use $K=30.0$ and $R_0=1.5$ which are standard parameters for modelling polymer systems. 

In [ ]:
fene = hoomd.md.bond.FENEWCA()
fene.paramas['A-A'] = dict(k=30.0, r0=1.5, sigma=1, epsilon=1)
integrator.forces.append(fene)

Now we can set up our integration method. When discussing polymers, we are usually referring to polymers in some solvent. There are ways to model solvent molecules explicitly using MD, but for our purposes (and most coarse-grained simulations) we don't need the explicit solvent molecules, we can instead model the solvent _implicitly_ using a friction (or drag) force in addition to random forces acting on the particles. 

This is called Langevin dynamics (or Brownian dynamics, which is just Langevin without inertia), and is a subset of Molecular Dynamics simulations. To implement Langevin Dynamics, we integrate the Langevin equations of motion in the canonical ensemble (NVT) as follows:

$$ m\frac{d\vec{v}}{dt} = \vec{F}_C - \gamma \cdot \vec{v} + \vec{F}_R$$
$$\langle{\vec{F}_R\rangle}=0$$
$$\langle{|\vec{F}_R|^2\rangle}=2dkT\gamma/\delta t$$

where $\vec{F}_C$ is the combined force on the particle from all potentials, $\gamma$ is the drag coefficient, $\vec{v}$ is the particle velocity, $\vec{F}_R$ is a uniform random force, and $d$ is the dimensionality of the system (in our case, $d=3$).


In [ ]:
langevin = hoomd.md.methods.Langevin(filter=hoomd.filter.All(), kT=1.0)
integrator.forces.append(langevin)

Now we can add in our thermo compute and table logger like usual.

In [ ]:
thermo = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermo)

table_logger = hoomd.logging.Logger(categories=['scalar', 'string'])
table_logger.add(sim, quantities=['timestep', 'tps', 'walltime'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=10000), logger=table_logger)
sim.operations.writers.append(table)

Just like the last simulation you worked on, we need to resize the box to achieve our target density. We are representing a dense polymer melt in this simulation, so we want to set the *density* of the system (not volume fraction) to $\rho=0.85$. Aside from the change from volume fraction to density, the process remains the same. 

In [ ]:
ramp = hoomd.variant.Ramp(A=0, B=1, t_start=sim.timestep, t_ramp=1000)
initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)
rho = 0.85
final_box.volume = (sim.state.N_particles)/rho
box_resize = hoomd.update.BoxResize(box1=initial_box,
                                    box2=final_box,
                                    variant=ramp,
                                    trigger = hoomd.trigger.Periodic(10))
sim.operations.updaters.append(box_resize)
sim.run(1000)
sim.operations.updaters.remove(box_resize)

Since we're dealing with polymer chains, however, we want to make sure that they're properly equilibrated by running the simulation for a little bit longer before we collect any data. 

In [ ]:
sim.run(5000)

Now we can collect our data! We're going to be computing MSDs later, so don't forget to include image flags.

In [ ]:
logger = hoomd.logging.Logger()
logger.add(thermo)
gsd_writer = hoomd.write.GSD(filename=f'log-chl{chl}.gsd', trigger=hoomd.trigger.Periodic(1000), mode='wb', 
                             filter=hoomd.filter.All(), dynamic=['particles/image'], logger=logger)
sim.operations.writers.append(gsd_writer)

sim.run(500000)
gsd_writer.flush()

Now that you've learned how to simulate a simple coarse-grained polymer melt, use this to simulate polymer melts with chain lengths of **2, 4, 6, 8, 12, and 16** and $N=480$ total particles in the system. For each of these, calculate the mean-squared displacement (follow the methods we used in HW3) and the diffusion constant. **Answer the following questions:**
1. What is coarse-graining, and why is it used in simulations? What are some of the important attributes of polymers that we want to make sure are reflected in coarse-grained models?
2. Plot all the MSDs on one figure. What trend do you notice in the MSD as chain length varies? Why does this trend occur? 
3. Plot the diffusion constant versus chain length. Does the diffusion constant appear to obey any obvious scaling law, $D\sim M^v$ where $v$ is an exponent?
4. Simulate chain lengths 4, 8, and 16 at five different temperatures, $T = 1.0, 1.5, 2.0, 2.5,$ and $3.0$ and calculate the diffusion coefficient for each. Plot $ln(D/T)$ as a function of $1/T$. To what extent does the diffusion constant follow an Arhennius relationship, $D\sim Texp(-E_a/k_bT)$? 

**Please submit the plots and explainations in one PDF document to Canvas!**

I recommend (though it's not required) attempting to automate this problem using python functions. One way to do this is by making a simulation function and an analysis function (you could also do this to generate the initial configuration files), then using a for loop to sweep the parameters of interest. If you are struggling with figuring out functions, check out [this link](https://www.w3schools.com/python/python_functions.asp) or come to my office hours!